In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp

import pytry

In [2]:
from sklearn.decomposition import PCA

class PatternInterpolationTrial(pytry.Trial):
    def params(self):
        self.param('number of neurons', n_neurons=2000),
        #self.param('number of training patterns', n_train_pattern=3)
        
    def evaluate(self, q):
        data = pd.read_csv("pinsoro-2017-06-20-145454904791-small.csv", low_memory=False)
        
        x = np.array(data.iloc[:,11:195]).astype(float)
        a = []
        for i in range(x.shape[1]):
            y = pd.Series(x[:,i])
            z = y.interpolate(limit_direction='both')
            a.append(z)
        a = pd.DataFrame(a)
        a = a.dropna()
        a = np.array(a).T

        dt = 0.001
        
        def extract_pattern(start, end):
            pattern = np.array(a[start:end,:]).astype(float)
            frames = np.array(data.iloc[start:end,9]).astype(int)

            good_indices = frames != -1
            frames = frames[good_indices]
            pattern = pattern[good_indices]

            fps = 30.0
            t_sample = (frames - frames[0])/fps

            t = np.arange(int(t_sample[-1]/dt))*dt

            result = []
            for i in range(pattern.shape[1]):       
                p = np.interp(t, t_sample, pattern[:,i])
                result.append(p)
            result = np.array(result).T

            return t, result


        t1, result1 = extract_pattern(928, 1222) #goaloriented
        t2, result2 = extract_pattern(1453, 1657)
        t3, result3 = extract_pattern(2076, 2302)
        t4, result4 = extract_pattern(2572, 2882)
        t5, result5 = extract_pattern(3818, 4843)
        t6, result6 = extract_pattern(5011, 5164)
        t7, result7 = extract_pattern(5762, 5843)
        t8, result8 = extract_pattern(6375, 6762)
        t9, result9 = extract_pattern(6927, 6981)
        t10, result10 = extract_pattern(8991, 9357)
        t11, result11 = extract_pattern(10175, 10280)
        t12, result12 = extract_pattern(11260, 11675)
        t13, result13 = extract_pattern(13811, 14582)
        t14, result14 = extract_pattern(18868, 19945)
        t15, result15 = extract_pattern(20255, 20979)
        t16, result16 = extract_pattern(22753, 22853)
        t17, result17 = extract_pattern(26263, 26604)
        t18, result18 = extract_pattern(26617, 26983)
        t19, result19 = extract_pattern(30051, 30419)
        t20, result20 = extract_pattern(30596, 30980)
        t21, result21 = extract_pattern(31145, 32070)
        
        
        t22, result22 = extract_pattern(91, 927) #noplay
        t23, result23 = extract_pattern(1223, 1452)
        t24, result24 = extract_pattern(1658, 2075)
        t25, result25 = extract_pattern(2303, 2571)
        t26, result26 = extract_pattern(2883, 3817)
        t27, result27 = extract_pattern(4844, 5010)
        t28, result28 = extract_pattern(5165, 5761)
        t29, result29 = extract_pattern(5844, 6374)
        t30, result30 = extract_pattern(6763, 6926)
        t31, result31 = extract_pattern(6982, 8683)
        t32, result32 = extract_pattern(9358, 9696)
        t33, result33 = extract_pattern(9796, 10174)
        t34, result34 = extract_pattern(10281, 10463)
        t35, result35 = extract_pattern(10823, 11259)
        t36, result36 = extract_pattern(11676, 13061)
        t37, result37 = extract_pattern(14583, 14991)
        t38, result38 = extract_pattern(15366, 15642)
        t39, result39 = extract_pattern(17134, 17978)
        t40, result40 = extract_pattern(18328, 18702)
        t41, result41 = extract_pattern(19946, 20254)
        t42, result42 = extract_pattern(21217, 22752)
        t43, result43 = extract_pattern(22854, 23559)
        t44, result44 = extract_pattern(23902, 24419)
        t45, result45 = extract_pattern(24982, 25610)
        t46, result46 = extract_pattern(26984, 27768)
        t47, result47 = extract_pattern(28268, 29387)
        t48, result48 = extract_pattern(29975, 30050)
        t49, result49 = extract_pattern(30420, 30595)
        t50, result50 = extract_pattern(30981, 31144)
        t51, result51 = extract_pattern(32071, 33540)        

        goal_list = [result1, result2, result3, result4, result5, result6, result7, result8, result9, result10, 
                    result11, result12, result13, result14, result15, result16, result17, result18, result19, 
                    result20, result21]
        noplay_list = [result22, result23, result24, result25, result26, result27, result28, result29, 
                      result30, result31, result32, result33, result34, result35, result36, result37, 
                      result38, result39, result40, result41, result42, result43, result44, result45, 
                      result46, result47, result48, result49, result50, result51]
        
        random.shuffle(goal_list)
        goal_train = goal_list[:15]
        goal_test = goal_list[15:]
        
        random.shuffle(noplay_list)
        noplay_train = noplay_list[:22]
        noplay_test = noplay_list[22:]
        
        train_all = np.vstack(goal_train+noplay_train)
        pca_model = PCA(n_components=1).fit(train_all)

        goal_patterns = {}
        i=0
        for gp in goal_train:
            goal_patterns[i] = pca_model.transform(gp)[:,0]
            i+=1

        noplay_patterns = {}
        i=0
        for npp in noplay_train:
            noplay_patterns[i] = pca_model.transform(npp)[:,0]
            i+=1

        goal_patterns = [(goal_patterns[key]) for key in goal_patterns.keys()]
        noplay_patterns = [(noplay_patterns[key]) for key in noplay_patterns.keys()] 

        theta = 1.0
        net = nengo.Network()
        with net:
            process = nengo.processes.WhiteSignal(period=100., high=1.0, y0=0)
            rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=q.n_neurons, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))
            
        s_window = int(theta/dt)
        s_pattern = min(len(goal_patterns[0]), len(goal_patterns[1]), len(goal_patterns[2]), len(goal_patterns[3]),
                        len(goal_patterns[4]), len(goal_patterns[5]), len(goal_patterns[6]), len(goal_patterns[7]),
                        len(goal_patterns[8]), len(goal_patterns[9]), len(goal_patterns[10]), len(goal_patterns[11]), 
                        len(goal_patterns[12]), len(goal_patterns[13]), len(goal_patterns[14]),
                        len(noplay_patterns[0]), len(noplay_patterns[1]), len(noplay_patterns[2]), len(noplay_patterns[3]), 
                        len(noplay_patterns[4]), len(noplay_patterns[5]), len(noplay_patterns[6]), len(noplay_patterns[7]), 
                        len(noplay_patterns[8]), len(noplay_patterns[9]), len(noplay_patterns[10]), len(noplay_patterns[11]), 
                        len(noplay_patterns[12]), len(noplay_patterns[13]), len(noplay_patterns[14]), len(noplay_patterns[15]), 
                        len(noplay_patterns[16]), len(noplay_patterns[17]), len(noplay_patterns[18]), len(noplay_patterns[19]), 
                        len(noplay_patterns[20]), len(noplay_patterns[21]))

        t_window = np.linspace(0, 1, s_window)
        inv_basis = rw.inverse_basis(t_window)

        eval_points=[]
        target = []
        for i in range(s_pattern):
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[0], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[1], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[2], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[3], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[4], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[5], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[6], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[7], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[8], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[9], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[10], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[11], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[12], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[13], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns[14], i)[:s_window]))
            target.append([1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[0], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[1], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[2], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[3], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[4], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[5], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[6], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[7], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[8], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[9], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[10], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[11], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[12], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[13], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[14], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[15], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[16], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[17], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[18], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[19], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[20], i)[:s_window]))
            target.append([-1])
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns[21], i)[:s_window]))
            target.append([-1])

        eval_points = np.array(eval_points)
        
        with net:
            result = nengo.Node(None, size_in=1)
            nengo.Connection(rw.state, result,
                             eval_points=eval_points, scale_eval_points=True,
                             function=target, synapse=0.1)
            
        train_all =  np.hstack([goal_patterns[0], goal_patterns[1], goal_patterns[2], goal_patterns[3],
                               goal_patterns[4], noplay_patterns[0], noplay_patterns[1], noplay_patterns[2],
                               noplay_patterns[3], noplay_patterns[4]])
        presentation_time_test=30/len(train_all)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(train_all, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

############ EXTRACT TEST PATTERNS ############ 
        goal_test_patterns = {}
        i=0
        for gp in goal_test:
            goal_test_patterns[i] = pca_model.transform(gp)[:,0]
            i+=1

        noplay_test_patterns = {}
        i=0
        for npp in noplay_test:
            noplay_test_patterns[i] = pca_model.transform(npp)[:,0]
            i+=1

        patterns = {}
        patterns.update(goal_test_patterns)
        patterns.update(noplay_test_patterns)

        goal_test_patterns = [(goal_test_patterns[key]) for key in goal_test_patterns.keys()]
        noplay_test_patterns = [(noplay_test_patterns[key]) for key in noplay_test_patterns.keys()] 
        
        test_all =  np.hstack([goal_test_patterns[0], goal_test_patterns[1], goal_test_patterns[2], goal_test_patterns[3],
                               goal_test_patterns[4], goal_test_patterns[5],  
                               noplay_test_patterns[0], noplay_test_patterns[1], noplay_test_patterns[2], noplay_test_patterns[3], 
                               noplay_test_patterns[4], noplay_test_patterns[5], noplay_test_patterns[6], noplay_test_patterns[7]])
        presentation_time_test=30/len(test_all)
        
        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(test_all, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)
        
        
        ###################################################  
        
        gp1_len = int(len(goal_test_patterns[0])*presentation_time_test/dt)
        gp2_len = gp1_len+int(len(goal_test_patterns[1])*presentation_time_test/dt)
        gp3_len = gp2_len+int(len(goal_test_patterns[2])*presentation_time_test/dt)
        gp4_len = gp3_len+int(len(goal_test_patterns[3])*presentation_time_test/dt)
        gp5_len = gp4_len+int(len(goal_test_patterns[4])*presentation_time_test/dt)
        gp6_len = gp5_len+int(len(goal_test_patterns[5])*presentation_time_test/dt)

        mean_gp1 = np.mean(sim.data[p_result][0:gp1_len])       
        mean_gp2 = np.mean(sim.data[p_result][gp1_len:gp2_len])
        mean_gp3 = np.mean(sim.data[p_result][gp2_len:gp3_len])
        mean_gp4 = np.mean(sim.data[p_result][gp3_len:gp4_len])
        mean_gp5 = np.mean(sim.data[p_result][gp4_len:gp5_len])
        mean_gp6 = np.mean(sim.data[p_result][gp5_len:gp6_len])
        
        npp1_len = gp5_len+int(len(noplay_test_patterns[0])*presentation_time_test/dt)
        npp2_len = npp1_len+int(len(noplay_test_patterns[1])*presentation_time_test/dt)
        npp3_len = npp2_len+int(len(noplay_test_patterns[2])*presentation_time_test/dt)
        npp4_len = npp3_len+int(len(noplay_test_patterns[3])*presentation_time_test/dt)
        npp5_len = npp4_len+int(len(noplay_test_patterns[4])*presentation_time_test/dt)
        npp6_len = npp5_len+int(len(noplay_test_patterns[5])*presentation_time_test/dt)
        npp7_len = npp6_len+int(len(noplay_test_patterns[6])*presentation_time_test/dt)
        npp8_len = npp7_len+int(len(noplay_test_patterns[7])*presentation_time_test/dt)
        
        mean_npp1 = np.mean(sim.data[p_result][gp5_len:npp1_len])       
        mean_npp2 = np.mean(sim.data[p_result][npp1_len:npp2_len])
        mean_npp3 = np.mean(sim.data[p_result][npp2_len:npp3_len])
        mean_npp4 = np.mean(sim.data[p_result][npp3_len:npp4_len])
        mean_npp5 = np.mean(sim.data[p_result][npp4_len:npp5_len])
        mean_npp6 = np.mean(sim.data[p_result][npp5_len:npp6_len])
        mean_npp7 = np.mean(sim.data[p_result][npp6_len:npp7_len])
        mean_npp8 = np.mean(sim.data[p_result][npp7_len:npp8_len])
        
        
################# TEST ON AIMLESS PATTERNS #################
        t53, result53 = extract_pattern(8684, 8990) #aimless
        t54, result54 = extract_pattern(9697, 9795)
        t55, result55 = extract_pattern(10464, 10822)
        t56, result56 = extract_pattern(13062, 13810)
        t57, result57 = extract_pattern(14992, 15365)
        t58, result58 = extract_pattern(15643, 17133)
        t59, result59 = extract_pattern(17979, 18327)
        t60, result60 = extract_pattern(18703, 18867)
        t61, result61 = extract_pattern(20980, 21216)
        t62, result62 = extract_pattern(23560, 23901)
        t63, result63 = extract_pattern(24420, 24981)
        t64, result64 = extract_pattern(25611, 26262)
        t65, result65 = extract_pattern(27769, 28267)
        t66, result66 = extract_pattern(29388, 29974)

        testp1 = pca_model.transform(result53)[:,0]
        testp2 = pca_model.transform(result54)[:,0]
        testp3 = pca_model.transform(result55)[:,0]
        testp4 = pca_model.transform(result56)[:,0]
        testp5 = pca_model.transform(result57)[:,0]
        testp6 = pca_model.transform(result58)[:,0]
        testp7 = pca_model.transform(result59)[:,0]
        testp8 = pca_model.transform(result60)[:,0]
        testp9 = pca_model.transform(result61)[:,0]
        testp10 = pca_model.transform(result62)[:,0]
        testp11 = pca_model.transform(result63)[:,0]
        testp12 = pca_model.transform(result64)[:,0]
        testp13 = pca_model.transform(result65)[:,0]
        testp14 = pca_model.transform(result66)[:,0]
       
        p_all = np.hstack([testp1, testp2, testp3, testp4, testp5, testp6,
                          testp7, testp8, testp9, testp10, testp11, testp12, 
                          testp13, testp14])

        presentation_time_test=30/len(p_all)

        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(p_all, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

        aim1_len = int(len(testp1)*presentation_time_test/dt)
        aim2_len = aim1_len+int(len(testp2)*presentation_time_test/dt)
        aim3_len = aim2_len+int(len(testp3)*presentation_time_test/dt)
        aim4_len = aim3_len+int(len(testp4)*presentation_time_test/dt)
        aim5_len = aim4_len+int(len(testp5)*presentation_time_test/dt)
        aim6_len = aim5_len+int(len(testp6)*presentation_time_test/dt)
        aim7_len = aim6_len+int(len(testp7)*presentation_time_test/dt)
        aim8_len = aim7_len+int(len(testp8)*presentation_time_test/dt)
        aim9_len = aim8_len+int(len(testp9)*presentation_time_test/dt)
        aim10_len = aim9_len+int(len(testp10)*presentation_time_test/dt)
        aim11_len = aim10_len+int(len(testp11)*presentation_time_test/dt)
        aim12_len = aim11_len+int(len(testp12)*presentation_time_test/dt)
        aim13_len = aim12_len+int(len(testp13)*presentation_time_test/dt)
        aim14_len = aim13_len+int(len(testp14)*presentation_time_test/dt)

        mean_aim1 = np.mean(sim.data[p_result][0:aim1_len])       
        mean_aim2 = np.mean(sim.data[p_result][aim1_len:aim2_len])
        mean_aim3 = np.mean(sim.data[p_result][aim2_len:aim3_len])
        mean_aim4 = np.mean(sim.data[p_result][aim3_len:aim4_len])
        mean_aim5 = np.mean(sim.data[p_result][aim4_len:aim5_len])
        mean_aim6 = np.mean(sim.data[p_result][aim5_len:aim6_len])
        mean_aim7 = np.mean(sim.data[p_result][aim6_len:aim7_len])
        mean_aim8 = np.mean(sim.data[p_result][aim7_len:aim8_len])
        mean_aim9 = np.mean(sim.data[p_result][aim8_len:aim9_len])
        mean_aim10 = np.mean(sim.data[p_result][aim9_len:aim10_len])
        mean_aim11 = np.mean(sim.data[p_result][aim10_len:aim11_len])
        mean_aim12 = np.mean(sim.data[p_result][aim11_len:aim12_len])
        mean_aim13 = np.mean(sim.data[p_result][aim12_len:aim13_len])
        mean_aim14 = np.mean(sim.data[p_result][aim13_len:aim14_len])
        
        return dict(
            mean_goal1=mean_gp1,
            mean_goal2=mean_gp2,
            mean_goal3=mean_gp3,
            mean_goal4=mean_gp4,
            mean_goal5=mean_gp5,
            mean_goal6=mean_gp6,
            mean_noplay1=mean_npp1,
            mean_noplay2=mean_npp2,
            mean_noplay3=mean_npp3,
            mean_noplay4=mean_npp4,
            mean_noplay5=mean_npp5,
            mean_noplay6=mean_npp6,
            mean_noplay7=mean_npp7,
            mean_noplay8=mean_npp8,
            mean_aim1=mean_aim1,
            mean_aim2=mean_aim2,
            mean_aim3=mean_aim3,
            mean_aim4=mean_aim4,
            mean_aim5=mean_aim5,
            mean_aim6=mean_aim6,
            mean_aim7=mean_aim7,
            mean_aim8=mean_aim8,
            mean_aim9=mean_aim9,
            mean_aim10=mean_aim10,
            mean_aim11=mean_aim11,
            mean_aim12=mean_aim12,
            mean_aim13=mean_aim13,
            mean_aim14=mean_aim14
        )


In [3]:
#PatternInterpolationTrial().run(data_dir='debug')

In [4]:
for seed in range(20):
    for n_neurons in [200, 500, 1000, 2000, 5000]:
        PatternInterpolationTrial().run(seed=seed, n_neurons=n_neurons,
                                        data_dir='vary_neurons_childchild')


running PatternInterpolationTrial#20190128-223243-392225f1


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo\utils\numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengolib\signal\system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 19
n_neurons = 5000
mean_aim1 = -0.14192409206620027
mean_aim10 = -0.41440561122171377
mean_aim11 = -0.49089590773734565
mean_aim12 = -0.1742975671351105
mean_aim13 = -0.3717089174840828
mean_aim14 = -0.08905353071742243
mean_aim2 = 0.08428391902393248
mean_aim3 = -0.39690988757118983
mean_aim4 = -0.24252078447164524
mean_aim5 = -0.021491973757668575
mean_aim6 = -0.11793780914969139
mean_aim7 = -0.3308364042372547
mean_aim8 = -0.16220205741769977
mean_aim9 = -0.4682766033639129
mean_goal1 = -0.2980918298892461
mean_goal2 = -0.4528909358970044
mean_goal3 = -0.29846988545830727
mean_goal4 = 0.41309951903938497
mean_goal5 = -0.2579686022539446
mean_goal6 = -0.005087325395206588
mean_noplay1 = 0.21068972953494047
mean_noplay2 = 0.12149465515201623
mean_noplay3 = -0.30002520960279033
mean_noplay4 = 0.3768178151082415
mean_noplay5 = -0.2628298784097204
mean_noplay6 = 0.12556290704740697
mean_noplay7 = -0.18153422004382855
mean_noplay8 = -0.26637579595388844
